<a href="https://colab.research.google.com/github/Sudaiii/ML-Experiments/blob/main/pinguino.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Librerias

No todas se usan. Se tomo la colección de librerias del GitHub del curso: https://github.com/castudil/Machine-Learning/blob/main/S04-KNN/Iris-KNN.ipynb


In [ ]:
import pandas as pd # amnipulacion de datos
import numpy as np # parte numerica
import matplotlib.pyplot as plt # visualizacion grafica
import seaborn as sns # mejora graficas, paletas de colores
# sklearn kit cientifico con mnodulos de machile learning.
from sklearn.model_selection import train_test_split # divide conjuntos de entrenamiento y validacione
from sklearn.preprocessing import StandardScaler, LabelEncoder # preprocesamiento de datos, en particular estandarizacion de datos.
from sklearn.neighbors import KNeighborsClassifier # algoritmo knn propiamente tal
from sklearn.metrics import confusion_matrix # matrices de confusión para analizar los resultados de los clasificadores
from sklearn.metrics import f1_score # tuvimos una métrica en particular que se conoce como score f1
from sklearn.metrics import accuracy_score # esta es la métrica por excelencia, denominada certeza en español

# Preprocesamiento

En esta sección se observan los datos y preparan para usar.

Se cargan el data set de los pingüinos y observamos la estructura de la tabla.

In [ ]:
url = "https://drive.google.com/file/d/1-DaBCr64zZCgwxGbQKK-Aiw9Ejt4KoN8/view?usp=sharing"
url = "https://drive.google.com/uc?id=" + url.split('/')[-2]
penguin = pd.read_csv(url)
penguin.head()

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE


Datos nulos no son utiles para los modelos de Machine Learning. Por lo tanto, se debe verificar si hay datos que se tengan que modificar o eliminar.

In [ ]:
penguin.isnull().sum()

<!DOCTYPE html><html><head><meta name="google" content="notranslate"><meta http-equiv="X-UA-Compatible" content="IE=edge;"><style nonce="n3r3gpR9nTgzBnCnd2bCNw">@font-face{font-family:'Roboto';font-style:italic;font-weight:400;src:url(//fonts.gstatic.com/s/roboto/v18/KFOkCnqEu92Fr1Mu51xIIzc.ttf)format('truetype');}@font-face{font-family:'Roboto';font-style:normal;font-weight:300;src:url(//fonts.gstatic.com/s/roboto/v18/KFOlCnqEu92Fr1MmSU5fBBc9.ttf)format('truetype');}@font-face{font-family:'Roboto';font-style:normal;font-weight:400;src:url(//fonts.gstatic.com/s/roboto/v18/KFOmCnqEu92Fr1Mu4mxP.ttf)format('truetype');}@font-face{font-family:'Roboto';font-style:normal;font-weight:500;src:url(//fonts.gstatic.com/s/roboto/v18/KFOlCnqEu92Fr1MmEU9fBBc9.ttf)format('truetype');}@font-face{font-family:'Roboto';font-style:normal;font-weight:700;src:url(//fonts.gstatic.com/s/roboto/v18/KFOlCnqEu92Fr1MmWUlfBBc9.ttf)format('truetype');}</style><meta name="referrer" content="origin"><title>penguins_s

Si hay datos nulos. Por lo tanto, hay que escoger una forma de manejarlo.

Para este estudio, se eliminaran todos las tuplas con datos nulos.


In [ ]:
penguin = penguin.dropna()
penguin.isnull().sum()


<!DOCTYPE html><html><head><meta name="google" content="notranslate"><meta http-equiv="X-UA-Compatible" content="IE=edge;"><style nonce="n3r3gpR9nTgzBnCnd2bCNw">@font-face{font-family:'Roboto';font-style:italic;font-weight:400;src:url(//fonts.gstatic.com/s/roboto/v18/KFOkCnqEu92Fr1Mu51xIIzc.ttf)format('truetype');}@font-face{font-family:'Roboto';font-style:normal;font-weight:300;src:url(//fonts.gstatic.com/s/roboto/v18/KFOlCnqEu92Fr1MmSU5fBBc9.ttf)format('truetype');}@font-face{font-family:'Roboto';font-style:normal;font-weight:400;src:url(//fonts.gstatic.com/s/roboto/v18/KFOmCnqEu92Fr1Mu4mxP.ttf)format('truetype');}@font-face{font-family:'Roboto';font-style:normal;font-weight:500;src:url(//fonts.gstatic.com/s/roboto/v18/KFOlCnqEu92Fr1MmEU9fBBc9.ttf)format('truetype');}@font-face{font-family:'Roboto';font-style:normal;font-weight:700;src:url(//fonts.gstatic.com/s/roboto/v18/KFOlCnqEu92Fr1MmWUlfBBc9.ttf)format('truetype');}</style><meta name="referrer" content="origin"><title>penguins_s

Además, hay datos que son discretos. Estos sirven más como categorias que como medidas de escala. Estos datos en esta tabla son representados por strings, lo cual los modelos no pueden manejar de forma directa. Por lo tanto, aquellos datos que son string se convierten a categorias numericas.

In [ ]:
label_enconder = LabelEncoder()
penguin["island"] = label_enconder.fit_transform(penguin["island"])
penguin["sex"] = label_enconder.fit_transform(penguin["sex"])

penguin[["island", "sex"]][:]

KeyError: ignored

Ahora, se debe separar los datos de los label (puesto a que si el modelo tuviera el label, seria trampa).

Aca, "x" corresponde a los datos y "y" al label.

In [ ]:
x = penguin.iloc[:, 1:]
y = penguin.iloc[:, 0]

In [ ]:
x.head()

In [ ]:
y.head()

Luego, debemos dividir los datos de entrenamiento de los datos de prueba. Aca se utiliza una distribución de datos de prueba de 30%.

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=0, test_size=0.3)

In [ ]:
print(len(x_train))
print(len(y_train))
print(len(x_test))
print(len(y_test))

Finalmente, hay que normalizar los datos de forma de que una columna no tenga mas poder que otra sobre las predicciones.

In [ ]:
sc_x = StandardScaler()
x_train = sc_x.fit_transform(x_train)
x_test = sc_x.transform(x_test)

# Predicción

Para este estudio se hara uso del modelo de predicción KNN.



In [ ]:
knn = KNeighborsClassifier(n_neighbors=15, p=2,metric='euclidean')

A este modelo le entregamos los datos de entrenamiento.

In [ ]:
knn.fit(x_train, y_train)

Luego hacemos que este modelo prediga a que corresponden los datos de prueba.

In [ ]:
y_pred = knn.predict(x_test)
y_pred

Una forma de comprobar cuales datos fueron correctos y cuales no es con una confusion matrix, donde alinea las predicciones con lo que eran realmente.

In [ ]:
from sklearn.metrics import f1_score
cm = confusion_matrix(y_test, y_pred)
print (cm)

In [ ]:
sns.heatmap(cm, annot=True, cbar=None, cmap="Blues")
plt.title("Confusion Matrix"), plt.tight_layout()
plt.ylabel("True Class"), plt.xlabel("Predicted Class")
plt.show()

Finalmente, de estos datos se puede obsevar se obtiene el siguiente porcentaje de exito:

In [ ]:
print(accuracy_score(y_test, y_pred))

# Cross-validation

Una forma de validar el modelo que es un poco mas avanzada es mediante Cross-validation.

A este le especificamos 10 "folds" y obtenemos sus resultados.

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(knn, x, y, cv=10)

Finalmente, obtenemos el promedio de estos valores.

In [ ]:
scores.mean()